In [1]:
import os
import numpy as np
import wfdb
import neurokit2 as nk

from operator import attrgetter

from utils import (
    extract_rri_signal,
    get_records_id,
    get_values_within_intervals,
    resample_ms
)

AFDB_PATH = "mit-bih-normal-sinus-rhythm-database-1.0.0/files"

record_ids = get_records_id(AFDB_PATH)

rri_output = np.array([])

for record_id in record_ids:

    record_path = os.path.join(AFDB_PATH, record_id)

    ecg_signal, ecg_metadata = wfdb.rdsamp(record_path)
    signal_len = ecg_metadata['sig_len']

    sample, aux_note = attrgetter("sample", "aux_note")(
        wfdb.rdann(record_path, "atr")
    )
    nsrdb_intervals = [[0, signal_len - 1]]
    
    for lead_idx, lead_signal in enumerate(ecg_signal.T):
        nsrdb_signal = get_values_within_intervals(nsrdb_intervals, lead_signal)
        _, rpeaks = nk.ecg_peaks(nsrdb_signal, sampling_rate=ecg_metadata["fs"])
        ecg_rpeaks = rpeaks["ECG_R_Peaks"]

        rri_signal = extract_rri_signal(ecg_rpeaks, signal_len)
        rri_signal_ms = resample_ms(rri_signal, ecg_metadata["fs"])
        rri_output = np.concatenate((rri_signal_ms, rri_output))

directory = "./output"
if not os.path.exists(directory):
    os.makedirs(directory)

np.save(os.path.join(directory, "nsrdb.npy"), rri_output)